In [15]:
import os
import sys

# Add scripts directory to path
current_dir = os.getcwd()
scripts_dir = current_dir.replace("experiments", "train")
sys.path.append(scripts_dir)

In [16]:
import pandas as pd
from datetime import datetime
from config import PATH_TO_SAVE_MODELS
models_info = pd.read_csv(os.path.join(PATH_TO_SAVE_MODELS, "models_info.csv")).iloc[-1].to_dict()

# Get last_row 
models_info
last_date = models_info["last_date"] 
# Convert to datetime 
last_date = datetime.strptime(last_date, "%Y-%m-%d %H:%M:%S")
model_name = models_info["model_name"]


In [17]:
# Generate documentation before each function and method definition

In [18]:

from datetime import datetime

def get_time_position(date, model_last_date):
    # Count the number of times between the last date and the date we want (by 10 minutes)
    return (date - model_last_date).days * 144 + (date - model_last_date).seconds // 600

date = datetime(2020, 4, 8, 22, 50)

n_predictions = get_time_position(date, last_date)

In [19]:
import torch
from darts.models import TCNModel

# Load the model
model = TCNModel.load(os.path.join(PATH_TO_SAVE_MODELS, model_name))#, map_location="cpu")
#model.to_cpu()
# Make predictions
output = model.predict(n_predictions, num_loader_workers=7, verbose=True, batch_size=16)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

In [20]:
predict_df = output["ActivePower"].pd_dataframe().reset_index(drop=False)
predict_df["ds"] = predict_df["ds"].dt.strftime("%Y-%m-%d %H:%M")

# Plot the time series with plotly express (ActivePower column is the target)
import plotly.express as px

fig = px.line(predict_df, x='ds', y='ActivePower', title='Time Series on ActivePower over time')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()


In [21]:


predict_df.to_dict(orient="records")

[{'ds': '2020-03-31 00:00', 'ActivePower': 477.67244670569136},
 {'ds': '2020-03-31 00:10', 'ActivePower': 598.3159235832265},
 {'ds': '2020-03-31 00:20', 'ActivePower': 594.6062582562747},
 {'ds': '2020-03-31 00:30', 'ActivePower': 553.4827784282274},
 {'ds': '2020-03-31 00:40', 'ActivePower': 568.6457465452839},
 {'ds': '2020-03-31 00:50', 'ActivePower': 521.9380706192215},
 {'ds': '2020-03-31 01:00', 'ActivePower': 464.34102270991536},
 {'ds': '2020-03-31 01:10', 'ActivePower': 482.94195495788705},
 {'ds': '2020-03-31 01:20', 'ActivePower': 589.2054717830543},
 {'ds': '2020-03-31 01:30', 'ActivePower': 550.9221246716945},
 {'ds': '2020-03-31 01:40', 'ActivePower': 486.3101697782903},
 {'ds': '2020-03-31 01:50', 'ActivePower': 499.03266912126696},
 {'ds': '2020-03-31 02:00', 'ActivePower': 465.63010021551594},
 {'ds': '2020-03-31 02:10', 'ActivePower': 404.92497663772303},
 {'ds': '2020-03-31 02:20', 'ActivePower': 402.7876727739332},
 {'ds': '2020-03-31 02:30', 'ActivePower': 422.00